# Setting Up Your Lab Environment

This guide shows you how to setup a Hashi environment for testing features in Consul, Vault, and Nomad.

Things to note:
* Enterprise binaries need to be licensed - set in `docker-compose*.yml`
  * Prem images are pre-license.
  * Consul 1.9, Nomad 1.0, Vault 1.7 has a starter license of 6 hours.
  * Consul 1.10+, Nomad 1.1+, and Vault 1.8+ requires a license file or it won't start


## Prerequisites

### Set Key Variables for your environment

In [311]:
export CONSUL_DC=west
export COMPOSE_PROJECT_NAME=hashi
export COMPOSE_FILE=docker-compose-hashi.yml:docker-compose-proxy.yml

Create needed directories.

In [113]:
mkdir -p consul/config

In [118]:
mkdir -p consul/cert/{server,client}

## Consul

### Create Consul Configs

Generate encryption key for Gossip - UDP; same key for all agents

In [19]:
CONSUL_KEY=$(consul keygen) && echo $CONSUL_KEY

WS/1KjlJkRNwSvrv1TXRvYaouwEw5+x8IvYrg9+5PjE=


Sample Output: `qDOPBEr+/oUVeOFQOnVypxwDaHzLrD+lvjo5vCEBbZ0=`

### Create CA and Certs

Create Certificate Authority

In [61]:
consul tls ca create

/media/usb_128g/hc_demos-jupyter/HashiStack/consul/config /media/usb_128g/hc_demos-jupyter/HashiStack
==> Saved consul-agent-ca.pem
==> Saved consul-agent-ca-key.pem


Copy CA Public Key to shared `client` and `server` folders.

In [128]:
for dir in client server; do
cp -r consul-agent-ca.pem consul/cert/${dir}/
done

Create server certificate and move it to shared `server` folder.

In [157]:
consul tls cert create -server -dc ${CONSUL_DC}
mv ${CONSUL_DC}-server-consul-*.pem consul/cert/server/

==> WARNING: Server Certificates grants authority to become a
    server and access all state in the cluster including root keys
    and all ACL tokens. Do not distribute them to production hosts
    that are not server nodes. Store them as securely as CA keys.
==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-server-consul-0.pem
==> Saved west-server-consul-0-key.pem


Create client certificate and move it to shared `client` folder.

In [120]:
consul tls cert create -client -dc ${CONSUL_DC} && \
  mv ${CONSUL_DC}-client-consul-*.pem consul/cert/client

==> Using consul-agent-ca.pem and consul-agent-ca-key.pem
==> Saved west-client-consul-0.pem
==> Saved west-client-consul-0-key.pem


Create Core Consul config - Server

In [24]:
# for i in {0..5}; do
tee consul/config/server.hcl <<-EOF
# datacenter  = "dc1" # in CLI
# node_name   = "ConsulServer${i}" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"

encrypt     = "${CONSUL_KEY}"
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/${CONSUL_DC}-server-consul-0.pem"
key_file    = "/consul/cert/${CONSUL_DC}-server-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

# server           =  true # in CLI
ui               = true
bootstrap_expect = 3
retry_join  = [
  "consul-server-0",
  "consul-server-1",
  "consul-server-2"
]
EOF
# done

Create Core Consul config - Client

In [25]:
# for i in {0..5}; do
cat > consul/config/client.hcl <<-EOF
# datacenter  = "dc1" # in CLI
# node_name   = "ConsulServer${i}" # in CLI
bind_addr   = "0.0.0.0" #default
client_addr = "0.0.0.0" #default 127.0.0.1
data_dir    = "/consul/data"

encrypt     = "${CONSUL_KEY}"
ca_file     = "/consul/cert/consul-agent-ca.pem"
cert_file   = "/consul/cert/${CONSUL_DC}-client-consul-0.pem"
key_file    = "/consul/cert/${CONSUL_DC}-client-consul-0-key.pem"
verify_incoming = true
verify_outgoing = true
verify_server_hostname = true

ui               = true
retry_join  = [
  "consul-server-0",
  "consul-server-1",
  "consul-server-2"
]
EOF
# done

Create Consul config for misc features eg `acl`, `performance multiplier`, etc

In [158]:
cat > consul/config/acl.hcl << EOF
# acl = {
#   enabled = true
#   default_policy = "allow"
#   enable_token_persistence = true
# }
performance {raft_multiplier = 1}
EOF

### Consul docker-compose up

We will now bring up the three Consul servers and one client. You can use `--force-recreate` to have Docker recreate the containers.

In [306]:
#docker-compose config

networks:
  vpcbr:
    driver: bridge
    ipam:
      config:
      - subnet: 10.5.0.0/16
services:
  consul-agent-1:
    command: agent -datacenter=west -retry-join consul-server-0 -client 0.0.0.0 -node=App1
    container_name: consul-agent-1
    environment:
      CONSUL_LICENSE_PATH: /consul/cert/consul.hclic
    hostname: consul-agent-1
    image: hashicorp/consul-enterprise:1.9-ent
    networks:
      vpcbr:
        ipv4_address: 10.5.0.12
    volumes:
    - /media/usb_128g/hc_demos-jupyter/HashiStack/consul/cert/client:/consul/cert:rw
    - /media/usb_128g/hc_demos-jupyter/HashiStack/consul/config/client.hcl:/consul/config/client.hcl:rw
    - /consul/data
  consul-server-0:
    command: agent -datacenter=west -server -config-dir=/consul/config
    container_name: consul-server-0
    environment:
      CONSUL_LICENSE_PATH: /consul/cert/consul.hclic
    hostname: consul-server-0
    image: hashicorp/consul-enterprise:1.9-ent
    networks:
      vpcbr:
        ipv4_address: 10.5.0.2

In [ ]:
export CONSUL_DC=west
docker-compose \
  up --force-recreate -d \
  consul-server-0 consul-server-1 consul-server-2 consul-agent-1
  # -f docker-compose-hashi.yml \

Recreating consul-server-2 ... 
Recreating consul-server-0 ... 
Recreating consul-agent-1  ... 
Recreating consul-server-1 ... 


### Verify Consul

Quick check to make sure your Consul environment is running correctly.

In [309]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers
printf "\n#==> List services from Consul catalog\n"
consul catalog services

#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  7e1300bd-c6fd-cfca-1c92-f9fa13ee6287  10.5.0.2:8300  follower  true   3
consul-server-1  e42d4ebb-30ff-0567-003b-e9d51bf50029  10.5.0.3:8300  leader    true   3
consul-server-2  bdd2d25c-b1d5-c313-3d05-f47921e3ab02  10.5.0.4:8300  follower  true   3

#==> List services from Consul catalog
consul
nomad
nomad-client
vault


You should see something like the following.
```#==> List Members
Node             Address        Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301  alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301  alive   server  1.9.11+ent  2         west  <all>
```

* There should be three servers. `DC` should match

```
#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-2  08f89457-d9db-b025-c65e-185246fe577c  10.5.0.4:8300  leader    true   3
consul-server-1  f4c7057f-83ec-11ac-2027-ca85eccfce89  10.5.0.3:8300  follower  true   3
consul-server-0  2c965ad0-5042-424c-259c-a5781d001d28  10.5.0.2:8300  follower  true   3

* There should be a leader and two followers.

#==> List services from Consul catalog
consul
```

## Vault

### Create Vault Configs

In [240]:
# Create Vault Directories
for node in {1..5}; do
mkdir -p vault/config/vault_s${node}
mkdir -p vault/logs/vault_s${node}
done

In [250]:
# Create Vault Server Config
for i in {1..3}; do
cat > vault/config/vault_s${i}/server${i}.hcl <<-EOF
# Note: this file will be re-written by script
api_addr     = "http://10.5.0.10${i}:8200"
cluster_addr = "https://10.5.0.10${i}:8201"
disable_mlock = true

# Base Configuration
listener "tcp" {
  address = "0.0.0.0:8200"
  tls_disable = "true"
}

ui = "true"
log_level="INFO"

# Raft configuration
storage "raft" {
  path    = "/vault/file"
  node_id = "vault_s${i}"
  retry_join {
    leader_api_addr = "http://vault_s1:8200"
  }
  retry_join {
    leader_api_addr = "http://vault_s2:8200"
  }
  retry_join {
    leader_api_addr = "http://vault_s3:8200"
  }
}

service_registration "consul" {
  address = "consul-server-0:8500"
}
EOF
done

### Vault docker-compose up

In [298]:
# Restart Vault Cluster
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  vault_s1 vault_s2 vault_s3

Recreating vault_s1 ... 
Recreating vault_s2 ... mdone
Recreating vault_s3 ... 
eating vault_s3 ... done

### Init Vault `init.sh`

In [252]:
printf "Init vault_s1 \n"
export VAULT_ADDR=http://localhost:8200
# sleep 5
vault operator init -format=json -n 1 -t 1 > /tmp/vault.txt

export VAULT_TOKEN=$(cat /tmp/vault.txt | jq -r '.root_token')
printf "\nRoot VAULT TOKEN is: $VAULT_TOKEN \n"
printf "\n*** Please Run: export VAULT_TOKEN=${VAULT_TOKEN} \n"
sleep 5

Init vault_s1 

Root VAULT TOKEN is: s.jaSQdKm0Q6C0vQznCqVEtUC4 

*** Please Run: export VAULT_TOKEN=s.jaSQdKm0Q6C0vQznCqVEtUC4 


### Unseal Vault `unseal.sh`

In [264]:
export VAULT_ADDR=http://localhost:8200
export unseal_key=$(cat /tmp/vault.txt | jq -r '.unseal_keys_b64[0]')
printf "${unseal_key}\n"

for i in {1..3}; do
docker exec -i vault_s${i} sh <<EOM
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key}
EOM
done

gNbB3iXOkoKaTWZ0/+IdPAHFEyeO2Yrdx0r4NxIHAs8=
Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.7.5+ent
Storage Type                           raft
Cluster Name                           vault-cluster-e995a4d8
Cluster ID                             6a03657e-1963-38a6-e1de-3ba9552c70bd
HA Enabled                             true
HA Cluster                             https://10.5.0.102:8201
HA Mode                                standby
Active Node Address                    http://10.5.0.102:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   556
Raft Applied Index                     556
Key                     Value


In [153]:
export VAULT_TOKEN=$(cat /tmp/vault.txt | jq -r '.root_token')
vault token lookup

Key                 Value
---                 -----
accessor            DXe3jDdBtiJzqJYratIHqefG
creation_time       1637004624
creation_ttl        0s
display_name        root
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.BmIUWWzJYJ2UupjA0obtYkNl
meta                <nil>
num_uses            0
orphan              true
path                auth/token/root
policies            [root]
ttl                 0s
type                service


### Verify Vault

In [266]:
vault status

Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.7.5+ent
Cluster Name                           vault-cluster-e995a4d8
Cluster ID                             6a03657e-1963-38a6-e1de-3ba9552c70bd
HA Enabled                             true
HA Cluster                             https://10.5.0.102:8201
HA Mode                                standby
Active Node Address                    http://10.5.0.102:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   563
Raft Applied Index                     563


In [265]:
vault operator raft list-peers

Node        Address            State       Voter
----        -------            -----       -----
vault_s1    10.5.0.101:8201    follower    true
vault_s2    10.5.0.102:8201    leader      true
vault_s3    10.5.0.103:8201    follower    true


In [202]:
vault secrets list
vault read sys/license

Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.7.5+ent
HA Enabled         true
Error reading the raft cluster configuration: Error making API request.

URL: GET http://localhost:8200/v1/sys/storage/raft/configuration
Code: 503. Errors:

* Vault is sealed
Error listing secrets engines: Error making API request.

URL: GET http://localhost:8200/v1/sys/mounts
Code: 503. Errors:

* Vault is sealed
Error reading sys/license: Error making API request.

URL: GET http://localhost:8200/v1/sys/license
Code: 503. Errors:

* Vault is sealed


: 2

In [155]:
vault write sys/license text=@vault/config/vault.hclic

Success! Data written to: sys/license


In [273]:
vault secrets enable kv
# vault write kv/game/account username=foo password=bar

Success! Enabled the kv secrets engine at: kv/


In [274]:
vault secrets list

Path          Type         Accessor              Description
----          ----         --------              -----------
cubbyhole/    cubbyhole    cubbyhole_6eaa11f9    per-token private secret storage
identity/     identity     identity_afd3cddb     identity store
kv/           kv           kv_b050a90b           n/a
sys/          system       system_efd35649       system endpoints used for control, policy and debugging


## Vault Performance Nodes

optional - Vault Performance Nodes
* main difference here is that it does not auto-join

In [257]:
# Create Vault Server Config
for i in {4..5}; do
cat > vault/config/vault_s${i}/server${i}.hcl <<-EOF
# Note: this file will be re-written by script
api_addr     = "http://10.5.0.10${i}:8200"
cluster_addr = "https://10.5.0.10${i}:8201"
disable_mlock = true

# Base Configuration
listener "tcp" {
  address = "0.0.0.0:8200"
  tls_disable = "true"
}

ui = "true"
log_level="INFO"

# Raft configuration
storage "raft" {
  path    = "/vault/file"
  node_id = "vault_s${i}"
}

service_registration "consul" {
  address = "consul-server-0:8500"
}
EOF
done

### Vault Performance Nodes docker-compose up

In [258]:
# Restart Vault Cluster
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  vault_s4

Recreating vault_s1 ... 
Creating vault_s4   ... mdone
ting vault_s4   ... done

In [267]:
docker exec -i vault_s4 sh -s <<EOM
export VAULT_ADDR=http://127.0.0.1:8200
vault operator raft join -non-voter http://vault_s1:8200
EOM

Key       Value
---       -----
Joined    true


In [268]:
export unseal_key=$(cat /tmp/vault.txt | jq -r '.unseal_keys_b64[0]')
printf "${unseal_key}\n"

for i in {4..4}; do
docker exec -i vault_s${i} sh <<EOM
export VAULT_ADDR=http://localhost:8200
vault operator unseal ${unseal_key}
EOM
done

gNbB3iXOkoKaTWZ0/+IdPAHFEyeO2Yrdx0r4NxIHAs8=
Key                Value
---                -----
Seal Type          shamir
Initialized        true
Sealed             true
Total Shares       1
Threshold          1
Unseal Progress    0/1
Unseal Nonce       n/a
Version            1.7.5+ent
Storage Type       raft
HA Enabled         true


In [269]:
vault operator raft list-peers

Node        Address            State       Voter
----        -------            -----       -----
vault_s1    10.5.0.101:8201    follower    true
vault_s2    10.5.0.102:8201    leader      true
vault_s3    10.5.0.103:8201    follower    true
vault_s4    10.5.0.104:8201    follower    false


* `vault_s4` is not a voter.

## haproxy - Load Balancer

In this section, we will set up haproxy to provide performance and high-availability for Vault. Client requests sent to haproxy for Vault will treated the following way:

* `GET` requests will be round-robined to all Vault nodes
* Non-`GET` requests will be sent only to the active Vault node.

In [271]:
docker-compose -f docker-compose-hashi.yml \
  -f docker-compose-proxy.yml up --force-recreate -d \
  haproxy

Creating haproxy ... 
ting haproxy ... done

### Validate

In [19]:
export VAULT_TOKEN=$(cat /tmp/vault.txt | jq -r '.root_token')

Send POST (Write) request

In [275]:
curl -H "X-Vault-Token: ${VAULT_TOKEN}" \
  -X POST \
  -d '{"data":{"foo":"bar"}}' \
  http://127.0.0.1:18200/v1/kv/data/game/account | jq -c
docker logs haproxy 2>&1 | tail -n 1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22    0     0  100    22      0    107 --:--:-- --:--:-- --:--:--   107
10.5.0.1:47272 [16/Nov/2021:00:12:46.754] primary_cluster_api primary_cluster_active_api/vault-active 0/1/203 289 -- 1/1/0/0/0 0/0


This should go to active server from `primary_cluster_active_api` backend.
```
... primary_cluster_api primary_cluster_active_api/vault-active 0/1/252 389 -- 1/1/0/0/0 0/0
```

In [276]:
for i in {1..9}; do
curl -s -H "X-Vault-Token: ${VAULT_TOKEN}" \
  -X GET \
  http://127.0.0.1:18200/v1/kv/data/game/account | jq -c .data.data
docker logs haproxy 2>&1 | tail -n 1
done

{"foo":"bar"}
10.5.0.1:47534 [16/Nov/2021:00:13:15.847] primary_cluster_api vault_read/vault-any1 0/1/3 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47540 [16/Nov/2021:00:13:16.081] primary_cluster_api vault_read/vault-any2 0/1/3 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47548 [16/Nov/2021:00:13:16.315] primary_cluster_api vault_read/vault-any3 0/0/2 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47556 [16/Nov/2021:00:13:16.555] primary_cluster_api vault_read/vault-any1 0/1/3 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47562 [16/Nov/2021:00:13:16.787] primary_cluster_api vault_read/vault-any2 0/0/2 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47570 [16/Nov/2021:00:13:17.032] primary_cluster_api vault_read/vault-any3 0/1/3 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47580 [16/Nov/2021:00:13:17.280] primary_cluster_api vault_read/vault-any1 0/0/2 319 -- 1/1/0/0/0 0/0
{"foo":"bar"}
10.5.0.1:47590 [16/Nov/2021:00:13:17.519] primary_cluster_api vault_read/vault-any2 0/1/2 319 -- 1/1/0/0/0 0/0


Send GET (Read) request 

This should go to any server from `vault_read` backend
```
... primary_cluster_api vault_read/vault_s2 0/0/6 423 -- 1/1/0/0/0 0/0
```

### Reload haproxy

If you make changes to haproxy.cfg, you can reload haproxy.

In [183]:
docker kill -s HUP haproxy

haproxy


## Clean Up

If you are done with your tests, you might want to shut everything down to reduce your heating bills.

### docker-compose down - everything

In [192]:
CONSUL_DC=west
export COMPOSE_PROJECT_NAME=hashi
export COMPOSE_FILE=docker-compose-hashi.yml:docker-compose-proxy.yml

In [247]:
docker-compose down

Stopping vault_s3        ... 
Stopping vault_s2        ... 
Stopping vault_s1        ... 
Stopping consul-server-2 ... 
Stopping consul-server-1 ... 
Stopping consul-server-0 ... 
Stopping consul-agent-1  ... 
Removing vault_s3        ... mdone
Removing vault_s2        ... 
Removing vault_s1        ... 
Removing consul-server-2 ... 
Removing consul-server-1 ... 
Removing consul-server-0 ... 
Removing consul-agent-1  ... 
Removing network hashi_vpcbr2mdone


## DEBUGGING

### Review logs

Review consul logs - for docker

In [49]:
for i in {0..3}; do
printf "docker logs consul-server-${i}\n"
docker logs consul-server-${i} | { head ; tail -n 3;}
printf "\n"
done

docker logs consul-server-0
==> Starting Consul agent...
           Version: '1.9.11+ent'
           Node ID: '2c965ad0-5042-424c-259c-a5781d001d28'
         Node name: 'consul-server-0'
        Datacenter: 'west' (Segment: '<all>')
            Server: true (Bootstrap: false)
       Client Addr: [0.0.0.0] (HTTP: 8500, HTTPS: -1, gRPC: -1, DNS: 8600)
      Cluster Addr: 10.5.0.2 (LAN: 8301, WAN: 8302)
           Encrypt: Gossip: true, TLS-Outgoing: true, TLS-Incoming: true, Auto-Encrypt-TLS: false

2021-11-15T17:05:52.955Z [WARN]  agent: Check socket connection failed: check=default/_nomad-check-01bd2cc7d206e8b625724dc0222789c02df5c4af error="dial tcp 0.0.0.0:4648: connect: connection refused"
2021-11-15T17:05:52.955Z [WARN]  agent: Check is now critical: check=default/_nomad-check-01bd2cc7d206e8b625724dc0222789c02df5c4af
2021-11-15T17:05:52.957Z [WARN]  agent: Check is now critical: check=default/_nomad-check-30f7971e26a03d0e2767122f7edfaaca35f8e001

docker logs consul-server-1
==> Sta

In [130]:
cat > .env <<EOF
#COMPOSE_PROJECT_NAME=helloworld
APACHE_EXPOSED_PORT=80
APACHE_1_IP=10.5.0.11
APACHE_2_IP=10.5.0.22
HA_PROXY_IP=10.5.0.33
#NETWORK_SUBNET=10.5.0.0/24
EOF

## Appendix

### Addresses


| Name | Address | Description |
| :--- | --- | --- |
| Consul | http://192.168.17.101:8500 | Consul Dashboard
| Nomad | http://192.168.17.101:4646 | Nomad Dashboard
| Vault | http://192.168.17.101:8200 | Vault Dashboard
| haproxy stats | http://192.168.17.101:11936 | haproxy Consul Dashboard
| haproxy - Consul | http://192.168.17.101:18500 | haproxy Consul Dashboard
| haproxy - Nomad | http://192.168.17.101:14646 | haproxy Nomad Dashboard
| haproxy - Vault | http://192.168.17.101:18200 | haproxy Vault Dashboard

### Consul Auto Upgrade

Modify `docker-compose-hashi.yml`. For consul-server-3, 4, and 5, comment the image parameter for `1.9` and uncomment the one for `latest`.

```yaml
    # image: hashicorp/consul-enterprise:1.9-ent
    image: hashicorp/consul-enterprise:latest
```

Start consul-server-3 consul-server-4 consul-server-5

In [99]:
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  consul-server-3 consul-server-4 consul-server-5

Recreating consul-server-4 ... 
Recreating consul-server-3 ... 
Recreating consul-server-5 ... 
eating consul-server-4 ... done

Verify Consul

In [103]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers

#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-3  10.5.0.5:8301   alive   server  1.10.4+ent  2         west  <all>
consul-server-4  10.5.0.6:8301   alive   server  1.10.4+ent  2         west  <all>
consul-server-5  10.5.0.7:8301   alive   server  1.10.4+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  follower  false  3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  false  3
consul-serve

### Directory Structure

In [5]:
tree

.
├── README.md
├── apache
│   ├── 1
│   │   ├── Dockerfile
│   │   └── index.html
│   └── 2
│       ├── Dockerfile
│       └── index.html
├── consul
│   ├── cert
│   │   ├── client
│   │   │   ├── consul-agent-ca.pem
│   │   │   ├── west-client-consul-0-key.pem
│   │   │   └── west-client-consul-0.pem
│   │   └── server
│   │       ├── consul-agent-ca.pem
│   │       ├── consul.hclic
│   │       ├── west-server-consul-0-key.pem
│   │       └── west-server-consul-0.pem
│   ├── config
│   │   ├── acl.hcl
│   │   ├── server.hcl
│   │   ├── server0.hcl
│   │   ├── server1.hcl
│   │   ├── server2.hcl
│   │   ├── server4.hcl
│   │   └── server5.hcl
│   └── policies
├── consul-agent-ca-key.pem
├── consul-agent-ca.pem
├── docker
│   └── haproxy
│       └── haproxy.cfg
├── docker-compose-app.yml
├── docker-compose-consul-app.yml
├── docker-compose-hashi.yml
├── docker-compose-proxy.yml
├── docker-compose-scratch.yml
├── docker-compose.yml
├── grafana
│   ├── dashboards
│   │   ├── alerts.yaml


Sample Output for tree
<details><summary></summary>

```
.
├── README.md
├── consul
│   ├── cert
│   │   ├── client
│   │   │   ├── consul-agent-ca.pem
│   │   │   ├── west-client-consul-0-key.pem
│   │   │   └── west-client-consul-0.pem
│   │   └── server
│   │       ├── consul-agent-ca.pem
│   │       ├── consul.hclic
│   │       ├── west-server-consul-0-key.pem
│   │       └── west-server-consul-0.pem
│   ├── config
│   │   ├── acl.hcl
│   │   ├── server.hcl
│   │   ├── server0.hcl
│   │   ├── server1.hcl
│   │   ├── server2.hcl
│   │   ├── server4.hcl
│   │   └── server5.hcl
│   └── policies
├── consul-agent-ca-key.pem
├── consul-agent-ca.pem
├── docker
│   └── haproxy
│       └── haproxy.cfg
├── docker-compose-app.yml
├── docker-compose-consul-app.yml
├── docker-compose-hashi.yml
├── docker-compose-proxy.yml
├── docker-compose-scratch.yml
├── docker-compose.yml
├── grafana
│   ├── dashboards
│   │   ├── alerts.yaml
│   │   ├── consul-server-monitoring_rev3.json
│   │   ├── node-exporter-full_rev22.json
│   │   ├── rules.yaml
│   │   ├── tempo-operational.json
│   │   ├── tempo-reads.json
│   │   ├── tempo-resources.json
│   │   └── tempo-writes.json
│   └── provisioning
│       ├── dashboards
│       │   └── dashboards.yaml
│       └── datasources
│           └── datasource.yml
├── haproxy
│   ├── haproxy.cfg
│   └── haproxy.cfg.txt2
├── hashi_troubleshooting.ipynb
└── vault
    ├── config
    │   ├── vault_s1
    │   │   └── server1.hcl
    │   ├── vault_s2
    │   │   └── server2.hcl
    │   └── vault_s3
    │       └── server3.hcl
    └── logs
        ├── vault_s1
        ├── vault_s2
        ├── vault_s3
        └── vaults_s3
```
</details>

### Resources

* https://learn.hashicorp.com/tutorials/consul/deployment-guide


### Additional Consul Steps

Setup Consul environment variables - Notice that since TLS encryption is enabled, you will now need to use the server certificates to complete all other tasks.

In [ ]:
export CONSUL_CACERT=/etc/consul.d/consul-agent-ca.pem
export CONSUL_CLIENT_CERT=/etc/consul.d/<dc-name>-<server/ client>-consul-<cert-number>.pem
export CONSUL_CLIENT_KEY=/etc/consul.d/<dc-name>-<server/   client>-consul-<cert-number>-key.pem

### Consul Redundancy Zones

In [82]:
for i in {0..2}; do
docker exec -i consul-server-${i} sh <<EOM
cat > /consul/config/rz.hcl <<EOF
node_meta {
  zone = "zone${i}"
}
EOF
cat /consul/config/rz.hcl
consul reload
EOM
done

node_meta {
  zone = "zone0"
}
Configuration reload triggered
node_meta {
  zone = "zone1"
}
Configuration reload triggered
node_meta {
  zone = "zone2"
}
Configuration reload triggered


Update Consul autopilot configuration to reflect the `node_meta` configuration.

In [83]:
consul operator autopilot set-config -redundancy-zone-tag=zone

Configuration updated!


In [84]:
consul operator autopilot get-config

CleanupDeadServers = true
LastContactThreshold = 200ms
MaxTrailingLogs = 250
MinQuorum = 0
ServerStabilizationTime = 10s
RedundancyZoneTag = "zone"
DisableUpgradeMigration = false
UpgradeVersionTag = ""


Sample Output
```
CleanupDeadServers = true
LastContactThreshold = 200ms
MaxTrailingLogs = 250
MinQuorum = 0
ServerStabilizationTime = 10s
RedundancyZoneTag = "zone"   <==---
DisableUpgradeMigration = false
UpgradeVersionTag = ""
```

Create Core Consul config - Server

In [67]:
for i in {0..2}; do
cat > consul/config/rz-${i}.hcl <<-EOF
node_meta {
  zone = "zone${i}"
}
autopilot {
  redundancy_zone_tag = "zone"
}
EOF
done

Modify docker-compose-hashi.yml. For consul-server-3, 4, and 5, uncomment the image parameter for `1.9` and comment the one for `latest`.

```yaml
    image: hashicorp/consul-enterprise:1.9-ent
    # image: hashicorp/consul-enterprise:latest
```

Start consul-server-3 consul-server-4 consul-server-5

In [85]:
docker-compose -f docker-compose-hashi.yml up --force-recreate -d \
  consul-server-3 consul-server-4 consul-server-5

Creating consul-server-4 ... 
Creating consul-server-3 ... 
Creating consul-server-5 ... 
ting consul-server-4 ... done

Verify Consul

In [86]:
printf "#==> List Members\n"
consul members
# curl http://127.0.0.1:8500/v1/agent/members | jq -c .[]
printf "\n#==> List Raft Peers\n"
consul operator raft list-peers

#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-3  10.5.0.7:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-4  10.5.0.6:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-5  10.5.0.5:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  follower  true   3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  leader    true   3
consul-serve

Sample Output
```
#==> List Members
Node             Address         Status  Type    Build       Protocol  DC    Segment
consul-server-0  10.5.0.2:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-1  10.5.0.3:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-2  10.5.0.4:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-3  10.5.0.7:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-4  10.5.0.6:8301   alive   server  1.9.11+ent  2         west  <all>
consul-server-5  10.5.0.5:8301   alive   server  1.9.11+ent  2         west  <all>
App1             10.5.0.12:8301  alive   client  1.9.11+ent  2         west  <default>

#==> List Raft Peers
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  follower  true   3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  false  3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
```

* All the new servers, once started, are added to the datacenter as non-voters (`Voter` = `false`). You can reference the Voter column in the output to verify it.

#### Test fault tolerance

Stop one of the voters. We use `consul-server-1` from `zone1`.

In [87]:
docker stop consul-server-1

consul-server-1


Verify that the correspondent non-voter in its redundancy zone gets promoted as a voter as soon as the server gets declared unhealthy.

In [91]:
consul operator raft list-peers

Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3


```shell
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3   <==---
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
```

* `consul-server-4` from `zone1` is now a voter

Once `server-server-4` gets promoted as a voter you can start Consul on `consul-server-1` again and verify the one voter per redundancy zone rule is still respected.

In [92]:
docker start consul-server-1

consul-server-1


In [93]:
consul operator raft list-peers

Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  follower  false  3


```shell
Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-5  b9c2c042-b161-3085-f417-d5739e6cbb50  10.5.0.5:8300  follower  false  3
consul-server-4  21c6c18d-b89d-edca-4f8e-011cbb244036  10.5.0.6:8300  follower  true   3
consul-server-3  524afd65-feea-79f2-1431-09b6dbbb8c05  10.5.0.7:8300  follower  false  3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  follower  false  3   <==---
```

* `consul-server` is up as a `follower`, but is no longer a `voter`.

Stop consul-server-3 consul-server-4 consul-server-5

In [94]:
docker-compose -f docker-compose-hashi.yml stop \
  consul-server-3 consul-server-4 consul-server-5

Stopping consul-server-3 ... 
Stopping consul-server-5 ... 
Stopping consul-server-4 ... 
ping consul-server-5 ... done

In [98]:
consul operator raft list-peers

Node             ID                                    Address        State     Voter  RaftProtocol
consul-server-0  bf8054d2-57da-128c-1a0e-b81ab694105d  10.5.0.2:8300  leader    true   3
consul-server-2  3c60ff6c-3836-9f3a-627c-80370f71b172  10.5.0.4:8300  follower  true   3
consul-server-1  d7e82aa0-2fa0-9308-5970-44e839786d2b  10.5.0.3:8300  follower  true   3


## Debug - Network

In [39]:
docker exec -i consul-server-0 sh <<EOM
hostname
ping -qc 1 consul-server-1
ping -qc 1 consul-server-2
EOM

consul-server-0
PING consul-server-1 (10.5.0.3) 56(84) bytes of data.

--- consul-server-1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.260/0.260/0.260/0.000 ms
PING consul-server-2 (10.5.0.4) 56(84) bytes of data.

--- consul-server-2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.218/0.218/0.218/0.000 ms


### docker-compose restart

In [16]:
docker-compose -f docker-compose-hashi.yml restart

Restarting consul-server-0 ... 
Restarting consul-server-1 ... 
arting consul-server-0 ... done

### docker-compose down

In [2]:
docker-compose -f docker-compose-hashi.yml down

Removing consul-server-1 ... 
Removing consul-server-0 ... 
Removing network monitoring_vpcbre


### Restart Vault Cluster

In [70]:
docker-compose -f docker-compose-hashi.yml restart vault_s1 vault_s2 vault_s3

Restarting vault_s3 ... 
Restarting vault_s2 ... 
Restarting vault_s1 ... 
arting vault_s1 ... done

## Apps

In [59]:
docker-compose -f docker-compose-hashi.yml -f docker-compose-app.yml up -d

Pulling db (hashicorpdemoapp/product-api-db:v0.0.17)...
v0.0.17: Pulling from hashicorpdemoapp/product-api-db

0fa6465e: Pulling fs layer
899232ff: Pulling fs layer
0476b7ab: Pulling fs layer
470eb370: Pulling fs layer
db6dc34b: Pulling fs layer
17c35d15: Pulling fs layer
905f4f68: Pulling fs layer
0687a25d: Pulling fs layer
8fa32d04: Pulling fs layer
46436326: Pulling fs layer
ff7aa983: Pulling fs layer
8b11efac: Pulling fs layer
c1b58fc3: Pulling fs layer
9244105e: Pulling fs layer
Digest: sha256:c3e8ca2ba549ecbfe59a7d161995d844180b491e312695569b8e401ccdce5b74 1.043kB/1.043kBB
Status: Downloaded newer image for hashicorpdemoapp/product-api-db:v0.0.17
vault_s1 is up-to-date
consul-server-2 is up-to-date
consul-server-0 is up-to-date
Recreating db ... 
vault_s3 is up-to-date
vault_s2 is up-to-date
consul-server-1 is up-to-date
consul-agent-0 is up-to-date
eating db ... done

## Vault DB

In [60]:
# This script configures a Postgres Dynamic Database credential database for benchmarking
vault secrets enable database

vault write database/config/postgres \
  plugin_name=postgresql-database-plugin \
  allowed_roles="*" \
  connection_url="postgresql://{{username}}:{{password}}@db:5432/products?sslmode=disable" \
  username="postgres" \
  password="password"

vault write database/roles/benchmarking \
    db_name=postgres \
    creation_statements="CREATE ROLE \"{{name}}\" WITH LOGIN PASSWORD '{{password}}' VALID UNTIL '{{expiration}}'; \
        GRANT SELECT ON ALL TABLES IN SCHEMA public TO \"{{name}}\";" \
    default_ttl="24h" \
    max_ttl="48h"

vault read database/creds/benchmarking

Success! Enabled the database secrets engine at: database/
Success! Data written to: database/roles/benchmarking
Key                Value
---                -----
lease_id           database/creds/benchmarking/sSerDQhyjqbBD5LvqIRu5yPK
lease_duration     24h
lease_renewable    true
password           PGBO-kfIxvnas7OFNz-7
username           v-root-benchmar-ZTzumhwHC2WO7aXplc08-1636742131


Admin token (optional): You may prefer using an admin token instead of root (for example if you’re using an existing cluster). If so, create an admin token using the vault-admin.hcl policy file shown below. This admin policy is authored based on the Vault Policies guide.

In [ ]:
# Assuming that VAULT_TOKEN is set with root or higher Admin token
vault policy write learn-admin admin-policy.hcl
vault token create -policy=learn-adminexport
VAULT_TOKEN=<token-from-above command>
vault token lookup

In [111]:
consul members
consul operator raft list-peers
consul operator autopilot get-config
vault operator raft list-peers

Node            Address         Status  Type    Build   Protocol  DC   Segment
ConsulServer0   10.5.0.2:8301   alive   server  1.10.4  2         dc1  <all>
ConsulServer1   10.5.0.3:8301   alive   server  1.10.4  2         dc1  <all>
ConsulServer2   10.5.0.4:8301   alive   server  1.10.4  2         dc1  <all>
consul-agent-0  10.5.0.12:8301  alive   client  1.10.4  2         dc1  <default>
Node           ID                                    Address        State     Voter  RaftProtocol
ConsulServer2  09a93096-bcd7-2841-1bcd-3dcbdf2f4efb  10.5.0.4:8300  leader    true   3
ConsulServer1  6076ceb7-6135-7beb-ef21-ca31e271b705  10.5.0.3:8300  follower  true   3
ConsulServer0  b771e8f4-fbf8-bd7d-32ac-25389354602f  10.5.0.2:8300  follower  true   3
CleanupDeadServers = true
LastContactThreshold = 200ms
MaxTrailingLogs = 250
MinQuorum = 0
ServerStabilizationTime = 10s
RedundancyZoneTag = ""
DisableUpgradeMigration = false
UpgradeVersionTag = ""
Node        Address            State       Voter
--